In [1]:
import operator
import pdb
import pandas as pd
import numpy as np
import subprocess
import os
import ast
import time
import paramiko

def run_linux_command(command):
    p = subprocess.Popen(command, universal_newlines=True, shell=True, 
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    text = p.stdout.read()
    retcode = p.wait()
    return text

def list_activepods(podtype):
    cassandra_active_pods = {}
    pods = run_linux_command('kubectl get pods | grep ' + podtype).split('\n')
    for pod in pods:
        pod = pod.split(' ')[0]
        ifconfig_content = run_linux_command('kubectl exec -it '+pod+' ifconfig')
        ifconfig_content = ifconfig_content.split('\n')
        for ifconfig in ifconfig_content:
            if 'inet' in ifconfig:
                pod_ip = (ifconfig.strip().split(' ')[1])
                cassandra_active_pods[pod] = pod_ip
                break
    return cassandra_active_pods

cassandra_pods = {}
cassandra_active_pods = {}
user_pods = {}
user_active_pods ={}
os.chdir('/root/cassandra-stress')
print(run_linux_command('kubectl create -f cassandra.yaml'))
time.sleep(2)
print(run_linux_command('kubectl autoscale deployment cassandra --min=2 --max=3 --cpu-percent=80'))
podname = run_linux_command('kubectl get pods | grep cassandra').split(' ')[0]
parent_started_pod = podname
print(run_linux_command('kubectl exec '+ podname +' service ssh start'))
print(run_linux_command('kubectl create -f pod_test1.yaml')) 
print(run_linux_command('kubectl create -f pod_test2.yaml'))
print(run_linux_command('kubectl create -f pod_test3.yaml'))
print(run_linux_command('kubectl create -f pod_test4.yaml'))
ifconfig_content = run_linux_command('kubectl exec -it '+podname+' ifconfig')
ifconfig_content = ifconfig_content.split('\n')
for ifconfig in ifconfig_content:
    if 'inet' in ifconfig:
        pod_ip = (ifconfig.strip().split(' ')[1])
        break
cassandra_pods[podname] = pod_ip

users=[]
for user in run_linux_command('kubectl exec '+parent_started_pod+' cat /etc/passwd | grep test').strip().split('\n'):
    users.append(user.split(':')[0])    
users_thread_history={}
for user in users:
    users_thread_history[user] = []
disabled_users = []

deployment.apps "cassandra" created

deployment.apps "cassandra" autoscaled

Starting OpenBSD Secure Shell server: sshd.

deployment.apps "pod-test1" created

deployment.apps "pod-test2" created

deployment.apps "pod-test3" created

deployment.apps "pod-test4" created



In [2]:
#Autoscaling
print(subprocess.Popen(["kubectl", "exec" , parent_started_pod, "cassandra-stress", "write", "n=10000"], stdout=subprocess.PIPE))

cassandra_active_pods = list_activepods('cassandra')   
if len(cassandra_pods) < len(cassandra_active_pods):
    iterations = list(set(cassandra_active_pods.keys()) - set(cassandra_pods.keys()))
    for podname in iterations:
        print(subprocess.Popen(["kubectl", "exec" , podname, "cassandra-stress", "write", "n=10000"], stdout=subprocess.PIPE))
        print(run_linux_command('kubectl exec '+ podname +' service ssh start'))
        cassandra_pods[podname] = cassandra_active_pods[podname]

Starting OpenBSD Secure Shell server: sshd.



In [3]:
#Load distribution after autoscaling
for user in users:
    cassandra_pods_CPUusage={}
    for podname in cassandra_pods.keys():
        try:
            container_id = run_linux_command('kubectl get pods '+podname+' -o yaml | grep containerID').split('//')[1]
            cpu_usage = (float(run_linux_command("docker stats "+ container_id.strip() + " --no-stream | awk 'FNR == 2 {print $3}'").strip()[:-1]))
            cassandra_pods_CPUusage[podname] = cpu_usage
        except:
            pass
    selected_pod = ''
    for podname in cassandra_pods_CPUusage.keys():
        if cassandra_pods_CPUusage[podname] > 30:
            pass
        else:
            selected_pod = podname
            print('Request will be served from {0} because CPU usage= {1} %'.format(selected_pod,cassandra_pods_CPUusage[podname]))
            break

    if selected_pod != '':
        command=run_linux_command('kubectl exec ' + parent_started_pod + ' cat /home/'+user+'/cassandra-command')
        ssh = paramiko.SSHClient()
        ssh.load_system_host_keys()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(cassandra_pods[parent_started_pod], username=user, password=user)
        ssh.exec_command('echo "" > /home/'+user+'/cassandra-command')

        command = command.replace('\n',"")
        print(user ,':', command)
        if command.startswith('cassandra-stress read n='):
            ssh = paramiko.SSHClient()
            ssh.load_system_host_keys()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(cassandra_pods[selected_pod], username=user, password=user)
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
            time.sleep(5)
    else:
        print('The server is busy. {0} your request couldn\'t be fulfilled.'.format(user))

Request will be served from cassandra-7bb5c6cf9b-6hmv2 because CPU usage= 0.67 %
test1 : cassandra-stress read n=1000 -rate threads=50
Request will be served from cassandra-7bb5c6cf9b-c8f48 because CPU usage= 1.76 %
test2 : cassandra-stress read n=1000 -rate threads=50
Request will be served from cassandra-7bb5c6cf9b-c8f48 because CPU usage= 4.39 %
test3 : cassandra-stress read n=1000 -rate threads=50
Request will be served from cassandra-7bb5c6cf9b-6hmv2 because CPU usage= 0.61 %
test4 : cassandra-stress read n=1000 -rate threads=50


In [12]:
#Priority defining and resource limitting
premium_users=['test2','test3']
normal_users=list(set(users) - set(premium_users))
for user in users:
    cassandra_pods_CPUusage={}
    for podname in cassandra_pods.keys():
        try:
            container_id = run_linux_command('kubectl get pods '+podname+' -o yaml | grep containerID').split('//')[1]
            cpu_usage = (float(run_linux_command("docker stats "+ container_id.strip() + " --no-stream | awk 'FNR == 2 {print $3}'").strip()[:-1]))
            cassandra_pods_CPUusage[podname] = cpu_usage
        except:
            pass
    selected_pod = ''
    for podname in cassandra_pods_CPUusage.keys():
        if cassandra_pods_CPUusage[podname] > 30:
            pass
        else:
            selected_pod = podname
            print('Request will be served from {0} because CPU usage= {1} %'.format(selected_pod,cassandra_pods_CPUusage[podname]))
            break
    
    command=run_linux_command('kubectl exec ' + parent_started_pod + ' cat /home/'+user+'/cassandra-command')
    command = command.replace('\n',"")
    if selected_pod != '':
        print(user ,':', command,'\n')
        if command.startswith('cassandra-stress read n='):
            ssh = paramiko.SSHClient()
            ssh.load_system_host_keys()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(cassandra_pods[selected_pod], username=user, password=user)
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
            time.sleep(2)
    else:
        selected_pod = list(cassandra_pods_CPUusage.keys())[list(cassandra_pods_CPUusage.values()).index(min(cassandra_pods_CPUusage.values()))]
        if user in premium_users:
            print('Currently all servers are busy. Your request is fulfilled with lower value of thread rate.')
            print('Your request will be served from {0}.Your command is: {1}. \n'.format(selected_pod,command))
            if command.startswith('cassandra-stress read n='):
                ssh = paramiko.SSHClient()
                ssh.load_system_host_keys()
                ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh.connect(cassandra_pods[selected_pod], username=user, password=user)
                command = command.split('threads=')[0]+'threads='+str(5)
                ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
                print('Your request is fulfilled as -->',user ,':', command,'\n')
                time.sleep(2)
        else:
            print('The server is busy. {0} your request couldn\'t be fulfilled. You are normal user. Apply for premium subscription.'.format(user))
            

Request will be served from cassandra-7bb5c6cf9b-c8f48 because CPU usage= 0.46 %
test1 : cassandra-stress read n=1000 -rate threads=10 

Currently all servers are busy. Your request is fulfilled with lower value of thread rate.
Your request will be served from cassandra-7bb5c6cf9b-c8f48.Your command is: cassandra-stress read n=1000 -rate threads=30. 

Your request is fulfilled as --> test2 : cassandra-stress read n=1000 -rate threads=5 

Currently all servers are busy. Your request is fulfilled with lower value of thread rate.
Your request will be served from cassandra-7bb5c6cf9b-c8f48.Your command is: cassandra-stress read n=5000 -rate threads=60. 

Your request is fulfilled as --> test3 : cassandra-stress read n=5000 -rate threads=5 

Request will be served from cassandra-7bb5c6cf9b-6hmv2 because CPU usage= 0.59 %
test4 : cassandra-stress read n=7000 -rate threads=90 



In [19]:
#Disable the pod
imposed_thread_limit = 50
imposed_thread_violate_window = 4
violation=0
premium_users=['test2','test3']
normal_users=list(set(users) - set(premium_users))


for user in users:
    if len(users_thread_history[user]) >= imposed_thread_violate_window:
        for i in range(1,imposed_thread_violate_window+1):
            if users_thread_history[user][-i] >= imposed_thread_limit:
                violation = violation+1
        if violation == imposed_thread_violate_window:
            print('User {0} has violated the thread limit. Disabling him.'.format(user))
            violation = 0
            disabled_users.append(user)
   
    if user not in disabled_users:
        cassandra_pods_CPUusage={}
        for podname in cassandra_pods.keys():
            try:
                container_id = run_linux_command('kubectl get pods '+podname+' -o yaml | grep containerID').split('//')[1]
                cpu_usage = (float(run_linux_command("docker stats "+ container_id.strip() + " --no-stream | awk 'FNR == 2 {print $3}'").strip()[:-1]))
                cassandra_pods_CPUusage[podname] = cpu_usage
            except:
                pass
        selected_pod = ''
        for podname in cassandra_pods_CPUusage.keys():
            if cassandra_pods_CPUusage[podname] > 30:
                pass
            else:
                selected_pod = podname
                print('Request will be served from {0} because CPU usage= {1} %'.format(selected_pod,cassandra_pods_CPUusage[podname]))
                break

        command=run_linux_command('kubectl exec ' + parent_started_pod + ' cat /home/'+user+'/cassandra-command')
        command = command.replace('\n',"")
        users_thread_history[user].append(int(command.split('threads=')[1]))

        if selected_pod != '':
            print(user ,':', command,'\n')
            if command.startswith('cassandra-stress read n='):
                ssh = paramiko.SSHClient()
                ssh.load_system_host_keys()
                ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh.connect(cassandra_pods[selected_pod], username=user, password=user)
                ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
                time.sleep(2)
            else:
                print('Command syntax is wrong.')
        else:
            selected_pod = list(cassandra_pods_CPUusage.keys())[list(cassandra_pods_CPUusage.values()).index(min(cassandra_pods_CPUusage.values()))]
            if user in premium_users:
                print('Currently all servers are busy. Your request is fulfilled with lower value of thread rate.')
                print('Your request will be served from {0}.Your command is: {1}. \n'.format(selected_pod,command))
                if command.startswith('cassandra-stress read n='):
                    ssh = paramiko.SSHClient()
                    ssh.load_system_host_keys()
                    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                    ssh.connect(cassandra_pods[selected_pod], username=user, password=user)
                    comand = command.split('threads=')[0]+'threads='+str(5)
                    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
                    print('Your request is fulfilled as -->',user ,':', command,'\n')
                    time.sleep(2)
                else:
                    print('Command syntax is wrong.')
            else:
                print('The server is busy. {0} your request couldn\'t be fulfilled. You are normal user. Apply for premium subscription.'.format(user))
    else:
        print('{0} : You have been blocked due to violating the thread limit.'.format(user))
        command=run_linux_command('kubectl exec ' + parent_started_pod + ' cat /home/'+user+'/cassandra-command')
        command = command.replace('\n',"")
        users_thread_history[user].append(int(command.split('threads=')[1]))

Request will be served from cassandra-7bb5c6cf9b-56knn because CPU usage= 2.56 %
test1 : cassandra-stress read n=5000 -rate threads=10 

Request will be served from cassandra-7bb5c6cf9b-krlvt because CPU usage= 0.45 %
test2 : cassandra-stress read n=3000 -rate threads=30 

User test3 has violated the thread limit. Disabling him.
test3 : You have been blocked due to violating the thread limit.
User test4 has violated the thread limit. Disabling him.
test4 : You have been blocked due to violating the thread limit.


In [20]:
users_thread_history

{'test1': [10, 10, 10, 10, 10],
 'test2': [30, 30, 30, 30, 30],
 'test3': [60, 60, 60, 60, 60],
 'test4': [100, 100, 100, 100, 100]}

In [21]:
#Delete the pod forever if he is forcefully messing up
#Can also be done by limiting the access. Steps are given at https://www.ostechnix.com/how-to-limit-users-access-to-the-linux-system/
while True:
    connected_users = run_linux_command('kubectl exec  '+ parent_started_pod + ' pinky')
    connected_users = connected_users.strip().split('\n')
    for connected_user in connected_users:
        for disabled_user in list(set(disabled_users)):
            if disabled_user in connected_user:
                disabled_userIP = connected_user.split(' ')[-1]
                print('{0} has connected from IP: {1}.'.format(disabled_user,disabled_userIP))
                disabled_user_podname = run_linux_command('kubectl get pods -o wide | grep '+disabled_userIP)
                disabled_user_podname = disabled_user_podname.strip().split(' ')[0]
                print('{0} belongs to pod: {1}'.format(disabled_userIP,disabled_user_podname))
                for deployment in run_linux_command('kubectl get deployments').strip().split('\n'):
                    for deploymentid in deployment.split(' '):
                        if deploymentid.startswith('pod') and deploymentid in disabled_user_podname:
                            deploymentid = deploymentid.strip()
                            print(run_linux_command('kubectl delete deployments '+deploymentid))

test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
deployment.extensions "pod-test3" deleted

test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 has connected from IP: 172.17.0.9.
172.17.0.9 belongs to pod: pod-test3-7857bc69cf-vvz5g
test3 h

KeyboardInterrupt: 

In [22]:
#Syscalls --> Alert to admin
unique_cassandra_syscalls = ['0', '1', '13', '2', '202', '21', '228', '232', '270', '74', '3', '37', '4', '5', '72', '78', '8', '96']
import fileinput
import re
import glob
from multiprocessing import Process, Pipe
import sys
import pandas as pd

def grep(PAT, FILES):
    returned_grep = []
    for line in fileinput.input(glob.glob(FILES)):
        if re.search(PAT, line):
             returned_grep.append(line)
    return returned_grep

cassandra_containers = run_linux_command('docker ps --format "{{.ID}}: {{.Names}}" | grep k8s_cassandra').strip().split('\n')
cassandra_containerid = []
for container in cassandra_containers:
    cassandra_containerid.append(container.split(':')[0])

def collect_syscalls(mpPipe):
    # This will take a long time
    print("Syscalls collection started.")
    with open('syscall_run_cmd.sh','w') as fl:
        fl.write("perf record -e 'raw_syscalls:sys_enter' -g --exclude-perf -a  --call-graph dwarf,4096 -o sys_call_collected -G kubepods/" + qosClass_type + "/pod"+poduid+ "/"+ container_longID)
    os.system('sh syscall_run_cmd.sh')
    mpPipe.send("Done")
    
for container_name in cassandra_containerid:
    container_longID = subprocess.check_output(['docker', 'inspect', '--format="{{.Id}}"', container_name])
    container_longID = ast.literal_eval(container_longID[:-1].decode("utf-8") )
    pod_name = run_linux_command("kubectl get pod -o jsonpath='{range .items[?(@.status.containerStatuses[].containerID==\"docker://" + container_longID +"\")]}{.metadata.name}{end}'")
    run_linux_command('kubectl get pods '+pod_name + ' -o yaml > pod_yamlfile')
    qosClass_type = grep('qosClass','pod_yamlfile')
    poduid = grep('uid','pod_yamlfile')
    os.system('rm -rf pod_yamlfile')
    qosClass_type = ''.join(qosClass_type)
    qosClass_type = qosClass_type.lower()
    if 'besteffort' in qosClass_type:
        qosClass_type = 'besteffort'
    else:
        qosClass_type = 'burstable'
    poduid = poduid[1].split(':')[1].strip() 
    parent_conn, child_conn = Pipe()
    p = Process(target=collect_syscalls, args=(child_conn,))
    p.start()
    time.sleep(5)
    try:
        os.system('pkill -f perf')
    except:
        pass
    time.sleep(5)
    print("Syscalls collection completed.")
    os.system('perf script -i sys_call_collected > sys_call_numbers')
    
    os.system('rm -rf syscall_run_cmd.sh')
    os.system('rm -rf sys_call_collected')
    
    with open('sys_call_numbers', 'r') as fl:
        content = fl.readlines()
    os.system('rm -rf sys_call_numbers')
    if len(content) > 5:
        Syscalls_list = []
        for i in range(len(content)):
            try: 
                Syscalls_list.append((content[i].split('NR')[1]).split(' ')[1])

            except:
                pass

        current_container_syscall = np.unique(Syscalls_list)
        if len(set(current_container_syscall) - set(unique_cassandra_syscalls)) > 0:
            print('Alert!!!!: {0} syscalls have newly appeared for pod {1}. They are {2}.'.format(len(set(current_container_syscall) - set(unique_cassandra_syscalls)), pod_name, set(current_container_syscall) - set(unique_cassandra_syscalls)))
        else:
            print('No new syscalls appear for pod {0}. \n All the syscall are within {1}'.format(pod_name,unique_cassandra_syscalls))

Syscalls collection started.
Syscalls collection completed.
No new syscalls appear for pod cassandra-7bb5c6cf9b-56knn. 
 All the syscall are within ['0', '1', '13', '2', '202', '21', '228', '232', '270', '74', '3', '37', '4', '5', '72', '78', '8', '96']
Syscalls collection started.
Syscalls collection completed.
Alert!!!!: 20 syscalls have newly appeared for pod cassandra-7bb5c6cf9b-krlvt. They are {'33', '14', '24', '44', '16', '7', '61', '204', '273', '20', '60', '28', '9', '15', '10', '45', '233', '186', '231', '55'}.


In [35]:
#Rollback
cassandra_pod_packagelist = ['adduser', 'apt', 'base-files', 'base-passwd', 'bash', 'bsdutils', 'bzip2', 'ca-certificates', 'ca-certificates-java', 'cassandra', 'cassandra-tools', 'cgmanager', 'coreutils', 'dash', 'dbus', 'debconf', 'debian-archive-keyring', 'debianutils', 'diffutils', 'dirmngr', 'dmsetup', 'dpkg', 'e2fslibs', 'e2fsprogs', 'file', 'findutils', 'fontconfig-config', 'fonts-dejavu-core', 'gcc-6-base', 'gnupg', 'gnupg-agent', 'gpgv', 'grep', 'gzip', 'hostname', 'init-system-helpers', 'iproute2', 'iputils-ping', 'java-common', 'krb5-locales', 'libacl1', 'libapparmor1', 'libapt-pkg5.0', 'libassuan0', 'libattr1', 'libaudit-common', 'libaudit1', 'libavahi-client3', 'libavahi-common-data', 'libavahi-common3', 'libblkid1', 'libbsd0', 'libbz2-1.0', 'libc-bin', 'libc6', 'libcap-ng0', 'libcap2', 'libcap2-bin', 'libcgmanager0', 'libcomerr2', 'libcryptsetup4', 'libcups2', 'libdb5.3', 'libdbus-1-3', 'libdebconfclient0', 'libdevmapper1.02.1', 'libedit2', 'libelf1', 'libexpat1', 'libfdisk1', 'libffi6', 'libfontconfig1', 'libfreetype6', 'libgcc1', 'libgcrypt20', 'libgdbm3', 'libglib2.0-0', 'libglib2.0-data', 'libgmp10', 'libgnutls30', 'libgpg-error0', 'libgpm2', 'libgssapi-krb5-2', 'libhogweed4', 'libicu57', 'libidn11', 'libip4tc0', 'libjemalloc1', 'libjpeg62-turbo', 'libk5crypto3', 'libkeyutils1', 'libkmod2', 'libkrb5-3', 'libkrb5support0', 'libksba8', 'liblcms2-2', 'libldap-2.4-2', 'libldap-common', 'liblz4-1', 'liblzma5', 'libmagic-mgc', 'libmagic1', 'libmnl0', 'libmount1', 'libncurses5', 'libncursesw5', 'libnettle6', 'libnih-dbus1', 'libnih1', 'libnpth0', 'libnspr4', 'libnss3', 'libopts25', 'libp11-kit0', 'libpam-cap', 'libpam-modules', 'libpam-modules-bin', 'libpam-runtime', 'libpam-systemd', 'libpam0g', 'libpcre3', 'libpcsclite1', 'libperl5.24', 'libpng16-16', 'libprocps6', 'libpython-stdlib', 'libpython2.7-minimal', 'libpython2.7-stdlib', 'libreadline7', 'libsasl2-2', 'libsasl2-modules-db', 'libseccomp2', 'libselinux1', 'libsemanage-common', 'libsemanage1', 'libsepol1', 'libsmartcols1', 'libsqlite3-0', 'libss2', 'libssl1.0.2', 'libssl1.1', 'libstdc++6', 'libsystemd0', 'libtasn1-6', 'libtinfo5', 'libudev1', 'libustr-1.0-1', 'libuuid1', 'libwrap0', 'libx11-6', 'libx11-data', 'libxau6', 'libxcb1', 'libxdmcp6', 'libxext6', 'libxi6', 'libxml2', 'libxmuu1', 'libxrender1', 'libxtst6', 'login', 'lsb-base', 'mawk', 'mime-support', 'mount', 'multiarch-support', 'ncurses-base', 'ncurses-bin', 'ncurses-term', 'net-tools', 'netbase', 'ntp', 'openjdk-8-jre-headless', 'openssh-client', 'openssh-server', 'openssh-sftp-server', 'openssl', 'passwd', 'perl', 'perl-base', 'perl-modules-5.24', 'pinentry-curses', 'procps', 'python', 'python-minimal', 'python2.7', 'python2.7-minimal', 'readline-common', 'rename', 'sed', 'sensible-utils', 'sgml-base', 'shared-mime-info', 'systemd', 'systemd-shim', 'sysvinit-utils', 'tar', 'tcpd', 'tzdata', 'ucf', 'util-linux', 'vim', 'vim-common', 'vim-runtime', 'x11-common', 'xauth', 'xdg-user-dirs', 'xml-core', 'xxd', 'xz-utils', 'zlib1g']

for containerid in cassandra_containerid:
    crawler_command = ['docker', 'run', '--rm', '--privileged', '--net=host', '--pid=host', '-v', '/cgroup:/cgroup:ro', '-v', '/sys/fs/cgroup:/sys/fs/cgroup:ro', '-v', '/var/lib/docker:/var/lib/docker:ro', '-v', '/var/run/docker.sock:/var/run/docker.sock', '-v', '-it', 'crawler', '--crawlmode', 'OUTCONTAINER', '--features', 'package', '--crawlContainers', containerid]
    crawler_output = subprocess.check_output(crawler_command)
    crawler_output = crawler_output.decode("utf-8").split('\n')

    current_container_packages=[]
    for i,record in enumerate(crawler_output):
        if crawler_output[i].startswith('package'):
            current_container_packages.append(ast.literal_eval(crawler_output[i].split('\t')[1]))
    
    container_longID = subprocess.check_output(['docker', 'inspect', '--format="{{.Id}}"', containerid])
    container_longID = ast.literal_eval(container_longID[:-1].decode("utf-8") )
    pod_name = run_linux_command("kubectl get pod -o jsonpath='{range .items[?(@.status.containerStatuses[].containerID==\"docker://" + container_longID +"\")]}{.metadata.name}{end}'")
    
    if len(set(current_container_packages) - set(cassandra_pod_packagelist)) > 0:
        print('Pod {0} has detected new package. It needs to go rollback. The detected new packages are: {1} \n'.format(pod_name,set(current_container_packages) - set(cassandra_pod_packagelist)))
        print(run_linux_command('kubectl get pod ' + pod_name + ' -o yaml | kubectl replace --force -f -'))
    elif len(set(cassandra_pod_packagelist) - set(current_container_packages)) > 0:
        print('Pod {0} is missing few packages. It needs to go rollback. The missing packages are: {1} \n'.format(pod_name,set(cassandra_pod_packagelist) - set(current_container_packages)))
        print(run_linux_command('kubectl get pod ' + pod_name + ' -o yaml | kubectl replace --force -f -'))
    else:
        print('Pod {0} is normal. No action needed'.format(pod_name))

Pod cassandra-7bb5c6cf9b-2r4wp is normal. No action needed
Pod cassandra-7bb5c6cf9b-krlvt has detected new package. It needs to go rollback. The detected new packages are: {'manpages', 'linux-libc-dev', 'libitm1', 'libc-dev-bin', 'libc6-dev', 'libasan3', 'libtsan0', 'libquadmath0', 'libcilkrts5', 'libgcc-6-dev', 'libubsan0', 'libgomp1', 'libisl15', 'libmpc3', 'cpp', 'libmpx2', 'liblsan0', 'libmpfr4', 'gcc-6', 'cpp-6', 'liblocale-gettext-perl', 'libcc1-0', 'binutils', 'libatomic1', 'libstdc++-6-dev', 'gcc'} 

pod "cassandra-7bb5c6cf9b-krlvt" deleted
pod "cassandra-7bb5c6cf9b-krlvt" replaced



In [ ]:
def heap_size_formatting(max_heap):
    if 'M' in max_heap:
        max_heap = float(max_heap[:-1])
    elif 'G' in max_heap:
        max_heap = float(max_heap[:-1])*1024
    elif 'K' in max_heap:
        max_heap = float(max_heap[:-1])/1024
    return(max_heap)

test1pod = run_linux_command('kubectl get pods | grep test1').split(' ')[0]
test2pod = run_linux_command('kubectl get pods | grep test2').split(' ')[0]
#run_linux_command('ssh test1@'+pod_ip)
import paramiko
ssh = paramiko.SSHClient()
ssh.load_system_host_keys()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(pod_ip, username='test1', password='test1')
training_frames = []
for thread_count in range(5,100,10):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cassandra-stress read n=2000 -rate threads='+str(thread_count))
    #print(ssh_stdout.read())  #To display the output by running this comand.
    line_start = 0
    timeout = 10
    import time
    
    while line_start == 0:
        endtime = time.time() + timeout
        while not ssh_stdout.channel.eof_received:
            time.sleep(1)
            if time.time() > endtime:
                ssh_stdout.channel.close()
                break
        readrequest_stats = ssh_stdout.readlines()
        
        for i,line in enumerate(readrequest_stats):
            if line.startswith('type       total ops,    op/s,    pk/s,   row/s'):
                line_start = i
                time.sleep(1)
                break
    
    line_end = 0    
    for i,line in enumerate(readrequest_stats[line_start:]):
        if line.endswith('Results:'):
            line_end = i
            break

    if line_end == 0:
        line_end = len(readrequest_stats)

    readrequest_extract = []
    for record in readrequest_stats[line_start:line_end]:
        readrequest_extract.append(record.split(','))

    for i in range(len(readrequest_extract)):
        for j in range(len(readrequest_extract[i])):
            readrequest_extract[i][j] = readrequest_extract[i][j].strip()

    import pandas as pd
    df = pd.DataFrame.from_records(readrequest_extract[1:])
    df.drop(0, axis=1, inplace=True)
    readrequest_extract[0][0] = readrequest_extract[0][0][-9:]
    df.columns = readrequest_extract[0]

    df = df.convert_objects(convert_numeric=True)
    readrequest_frame = []
    for i in [1,4,5,6,7,8,9,11]:        
        readrequest_frame.append(round(np.mean(df[df.columns[i]]),5))

    for record in run_linux_command('cat etc/cassandra/cassandra-env.sh | grep MAX_HEAP_SIZE=').split('\n'):
        if record.startswith('MAX_HEAP_SIZE'):
            max_heap = ast.literal_eval(record.split('=')[1])
            max_heap = heap_size_formatting(max_heap)

    for record in run_linux_command('cat etc/cassandra/cassandra-env.sh | grep HEAP_NEWSIZE=').split('\n'):
        if record.startswith('HEAP_NEWSIZE='):
            new_heap = ast.literal_eval(record.split('=')[1])
            new_heap = heap_size_formatting(new_heap)    

    readrequest_frame.append(max_heap)
    readrequest_frame.append(new_heap)
    
    pod_containerid = run_linux_command('kubectl get pods cassandra-575c848f7c-xl946 -o yaml | grep containerID').split('//')[1].strip()
    pid_nums = float(run_linux_command("docker stats "+pod_containerid+" --no-stream | awk 'FNR == 2 {print $14}'").strip())
    readrequest_frame.append(pid_nums)
    
    print('thread_count=',thread_count)
    training_frames.append(readrequest_frame)

In [ ]:
#Parse the CPU, process, metric, connection feature
pod_containerid = 'fff5cc9ff1c4'
crawler_command = ['docker', 'run', '--rm', '--privileged', '--net=host', '--pid=host', '-v', '/cgroup:/cgroup:ro', '-v', '/sys/fs/cgroup:/sys/fs/cgroup:ro', '-v', '/var/lib/docker:/var/lib/docker:ro', '-v', '/var/run/docker.sock:/var/run/docker.sock', '-v', '-it', 'crawler', '--crawlmode', 'OUTCONTAINER', '--features', 'cpu,metric,process,connection', '--crawlContainers', pod_containerid]
crawler_output = subprocess.check_output(crawler_command)
crawler_output = crawler_output.decode("utf-8").split('\n')

dict_container_crawled = {}
cpu_usage = []
process_names = []
connection_names = []
metric_names = []

for i,record in enumerate(crawler_output):
    if crawler_output[i].startswith('cpu'):
        cpu_usage.append(crawler_output[i].split('\t')[2])

    if crawler_output[i].startswith('process'):
        process_names.append(crawler_output[i].split('\t')[1])

    if crawler_output[i].startswith('metric'):
        metric_names.append(crawler_output[i].split('\t')[2])

    if crawler_output[i].startswith('connection'):
        connection_names.append(crawler_output[i].split('\t')[2])

if ast.literal_eval(''.join(cpu_usage))['cpu_idle'] < 30:
    if 'cassandra-stres' in ''.join(process_names):
        agent_IPaddress = {}
        java_pids = []
        for record in connection_names:
            connection_dict = record.replace('null','0')
            connection_dict = ast.literal_eval(connection_dict)
            if connection_dict['connstatus'] == 'ESTABLISHED':
                if connection_dict['pname'] == 'sshd':
                    agent_IPaddress[connection_dict['pid']] = [connection_dict['remoteipaddr']]

                if connection_dict['pname'] == 'java':
                    java_pids.append(connection_dict['pid'])
    
    if len(agent_IPaddress) > 0:
        for key in agent_IPaddress.keys():
            for record in metric_names:
                metric = record.replace('null','0')
                metric = ast.literal_eval(metric)
                if metric['pid'] == key:
                    agent_IPaddress[key].append(metric['user'])

    cassandra_processdetails={}
    if len(java_pids) > 0:
        java_processdetails = {}
        for record in metric_names:
            metric = record.replace('null','0')
            metric = ast.literal_eval(metric)
            if metric['pname'] == 'java':
                java_processdetails[metric['pid']] = [metric['cpupct'],metric['mempct']]
                
            if metric['pname'] == 'cassandra-stres':
                cassandra_processdetails[metric['pid']] = metric['user']

In [ ]:
print('X=',np.array(training_frames))
print('Y=',[i for i in range(5,100,10)])

In [ ]:
for iteration,filename in enumerate(filenames):
    with open('/root/cassandra-stress/'+filename) as fl:
        logs = fl.readlines()

    thread_count = []
    for line in logs:
        if line.startswith('Running with '):
            thread_count.append(int(line.split(' ')[2]))
    #print thread_count
    complete_frame = []
    current_frame = []
    for index,line in enumerate(logs):
        if line.startswith('Results'):
            for i in range(1,17):
                extracted_part =  logs[index+i].split(':')[1].split('\t')[0].lstrip().split(' ')[0]
                if ',' in extracted_part: 
                    extracted_part = extracted_part.replace(',','')    
                current_frame.append(float(extracted_part))
            complete_frame.append(current_frame)
            current_frame = []

    complete_frame = pd.DataFrame(complete_frame)
    complete_frame.columns = ['op-rate','partition-rate','row-rate','latency-mean','latency-median','latency-95p','latency-99p','latency-99.99p','latency-max','total-partition','total-errors','total-gc-count','total-memory','total-gc-time','avg-gc-time','stddev-gc-time']
    complete_frame['heap-size'] = int(filename.split('_')[1].split('heap')[1][:-1]) * np.ones((len(complete_frame),1))
    complete_frame['Maxheap-size'] = int(filename.split('_')[2].split('max')[1].split('M')[0]) * np.ones((len(complete_frame),1))
    complete_frame['thread-count'] = thread_count
    if iteration != 0:
        complete_frame = pd.concat([lastiter_frame, complete_frame],ignore_index=True)
    lastiter_frame = complete_frame    
complete_frame.drop('total-partition', axis=1, inplace=True)

for i in range(len(complete_frame['thread-count'])):
    if complete_frame['thread-count'][i] < 20:
        complete_frame['thread-count'][i] = 'Below 20'
    elif complete_frame['thread-count'][i] >= 20 and complete_frame['thread-count'][i] < 50:
        complete_frame['thread-count'][i] = '20-49'
    elif complete_frame['thread-count'][i] >= 50 and complete_frame['thread-count'][i] < 100:
        complete_frame['thread-count'][i] = '50-99'
    elif complete_frame['thread-count'][i] >= 100 and complete_frame['thread-count'][i] < 300:
        complete_frame['thread-count'][i] = '100-299'
    elif complete_frame['thread-count'][i] >= 300 and complete_frame['thread-count'][i] < 600:
        complete_frame['thread-count'][i] = '300-599'   
    else:
        complete_frame['thread-count'][i] = '600-and-above'

In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

X=complete_frame.values[:,:-1]
Y=complete_frame['thread-count']
cleanup_nums = {'100-299':0, '20-49':1, '300-599':2, '50-99':3, '600-and-above':4, 'Below 20':5}
Y.replace(cleanup_nums,inplace=True)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
from sklearn.linear_model import LogisticRegression 
logisticRegr = LogisticRegression(solver = 'liblinear',verbose=1)
logisticRegr.fit(X_train,Y_train)
score = logisticRegr.score(X_test , Y_test)
print(score)

# Make predictions on test data
predictions = logisticRegr.predict(X_test)
cm = metrics.confusion_matrix(Y_test, predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {:.3f}'.format(score) 
plt.show()



In [ ]:
complete_frame.columns

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

import tensorflow as tf
slim = tf.contrib.slim
graph_replace = tf.contrib.graph_editor.graph_replace

import sys, os
sys.path.extend([os.path.expanduser('..')])
from pathint import utils
import seaborn as sns
sns.set_style("ticks")

from tqdm import trange, tqdm

# import operator
import matplotlib.colors as colors
import matplotlib.cm as cmx

rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

select = tf.select if hasattr(tf, 'select') else tf.where

# Data params
input_dim = 17
output_dim = 6

# Network params
n_hidden_units = 51
activation_fn = tf.nn.relu

# Optimization params
batch_size = 10
epochs_per_task = 10

n_stats = 10

# Reset optimizer after each age
reset_optimizer = True

from keras.utils import np_utils

#task_labels = [[0,1], [2,3], [4,5], [6,7], [8,9],[1,5],[7,9],[3,8],[0,6],[4,2]]
#task_labels = [[8,9], [6,7], [4,5], [2,3], [0,1]]
#task_labels = [[1,4], [0,9], [7,8], [3,6], [2,5]]
#task_labels = [[0,1,2], [3,4,5], [6,7,8,9]]
#task_labels = [[1,5,8],[2,5,7,9],[3,4,6]]
task_labels = [[0,1], [2,3], [4,5]]
n_tasks = len(task_labels)
nb_classes  = 6
training_datasets = []
validation_datasets = []
multihead=False

for labels in task_labels:
    idx = np.in1d(Y_train, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = X_train[idx], np_utils.to_categorical(label_map[Y_train[idx]], len(labels))
    else:
        data = X_train[idx], np_utils.to_categorical(Y_train[idx], nb_classes)
        training_datasets.append(data)

for labels in task_labels:
    idx = np.in1d(Y_test, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = X_test[idx], np_utils.to_categorical(label_map[Y_test[idx]], len(labels))
    else:
        data = X_test[idx], np_utils.to_categorical(Y_test[idx], nb_classes)
        validation_datasets.append(data)
        
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())
import keras.backend as K
import keras.activations as activations

output_mask = tf.Variable(tf.zeros(output_dim), name="mask", trainable=False)

def masked_softmax(logits):
    # logits are [batch_size, output_dim]
    x = select(tf.tile(tf.equal(output_mask[None, :], 1.0), [tf.shape(logits)[0], 1]), logits, -1e32 * tf.ones_like(logits))
    return activations.softmax(x)

def set_active_outputs(labels):
    new_mask = np.zeros(output_dim)
    for l in labels:
        new_mask[l] = 1.0
    sess.run(output_mask.assign(new_mask))
    sess.run(output_mask)
    
def masked_predict(model, data, targets):
    pred = model.predict(data)
    #print(pred)
    acc = np.argmax(pred,1)==np.argmax(targets,1)
    return acc.mean()

from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(n_hidden_units, kernel_initializer='random_uniform', activation=activation_fn, input_shape=(input_dim,)))
model.add(Dense(n_hidden_units, kernel_initializer='random_uniform', activation=activation_fn))
#model.add(Dense(n_hidden_units, activation=activation_fn))
model.add(Dense(output_dim, kernel_initializer='random_uniform', activation=masked_softmax))

from pathint import protocols
from pathint.optimizers import KOOptimizer
from keras.optimizers import Adam, RMSprop,SGD
from keras.callbacks import Callback
from pathint.keras_utils import LossHistory
from keras.callbacks import History 
from keras.callbacks import LambdaCallback

#protocol_name, protocol = protocols.PATH_INT_PROTOCOL(omega_decay='sum',xi=1e-3)
protocol_name, protocol = protocols.PATH_INT_PROTOCOL(omega_decay='sum',xi=1e-3)
#protocol_name, protocol = protocols.FISHER_PROTOCOL('sum')
opt = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
#opt = SGD(1e-3)
#opt = RMSprop(lr=1e-3)
oopt = KOOptimizer(opt, model=model, **protocol)
model.compile(loss='categorical_crossentropy', optimizer=oopt, metrics=['accuracy'])
model.model._make_train_function()
saved_weights = model.get_weights()

save_weights_epoch=[]
save_loss_epoch=[]
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: save_weights_epoch.append(model.get_weights()))
history = LossHistory()
#history = History()
callbacks = [history]
datafile_name = "split_mnist_data_%s.pkl.gz"%protocol_name

print(protocol)

def run_fits(cvals, training_data, valid_data, eval_on_train_set=False, nstats=1):
    acc_mean = dict()
    acc_std = dict()
    model_weights_save = []   #Empty list to save the model weights aftertraining each task
    for cidx, cval_ in enumerate(tqdm(cvals)):
        runs = []
        for runid in range(nstats):
            sess.run(tf.global_variables_initializer())
            # model.set_weights(saved_weights)
            cstuffs = []
            evals = []
            print("setting cval")
            cval = cval_
            oopt.set_strength(cval)
            oopt.init_task_vars()
            print("cval is", sess.run(oopt.lam))
            for age, tidx in enumerate(range(n_tasks)):
                print("Age %i, cval is=%f"%(age,cval))
                print("settint output mask")
                set_active_outputs(task_labels[age])
                stuffs = model.fit(training_data[tidx][0], training_data[tidx][1], batch_size, epochs_per_task, callbacks=[print_weights])
                save_loss_epoch.append(stuffs.history['loss'])
                #if age != 0:
                    #model_prune(age, unimportant_threshold = 5)
                oopt.update_task_metrics(training_data[tidx][0], training_data[tidx][1], batch_size)
                oopt.update_task_vars()
                ftask = []
                model_weights_save.append(model.get_weights()) #Save the model weights aftertraining each task
                for j in range(n_tasks):
                    set_active_outputs(task_labels[j])
                    if eval_on_train_set:
                        f_ = masked_predict(model, training_data[j][0], training_data[j][1])
                    else:
                        f_ = masked_predict(model, valid_data[j][0], valid_data[j][1])
                    ftask.append(np.mean(f_))
                evals.append(ftask)
                cstuffs.append(stuffs)

                # Re-initialize optimizater variables
                if reset_optimizer:
                    oopt.reset_optimizer()

            evals = np.array(evals)
            runs.append(evals)
        
        runs = np.array(runs)
        acc_mean[cval_] = runs.mean(0)
        acc_std[cval_] = runs.std(0)
    return dict(mean=acc_mean, std=acc_std),model_weights_save

cvals = [1.0]
print(cvals)

np.random.seed(0)
recompute_data = True

if recompute_data:
    data,model_weights_save = run_fits(cvals, training_datasets, validation_datasets, eval_on_train_set=False, nstats=n_stats)
    utils.save_zipped_pickle(data, datafile_name)



In [ ]:
import h5py
print(model.summary())
model.save_weights('saved_weights.h5') #This file cannot be opend normaly to view the weghts. It can be loaded through load_model() or can be opend via hdf5 viewer

#Shape of the array containg model weights
a_list = model.get_weights()
for i in range(len(a_list)):
    print((np.array(a_list[i])).shape)

In [ ]:
data = utils.load_zipped_pickle(datafile_name)
print(cvals)
for k in cvals:
    for i in range(n_tasks):
        for j in range(i):
            data['mean'][k][j][i] = 0
            data['std'][k][j][i] = 0
        
print(data)


In [ ]:
import matplotlib.colors as colors
cmap = plt.get_cmap('cool') 
cNorm  = colors.Normalize(vmin=-5, vmax=np.log(np.max(list(data['mean'].keys()))))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
print(scalarMap.get_clim())

figure(figsize=(2, 12))
axs = [subplot(n_tasks+1,1,1)]#, None, None]
for i in range(1, n_tasks + 1):
    axs.append(subplot(n_tasks+1,1, i+1, sharex=axs[0], sharey=axs[0]))
    
keys = list(data['mean'].keys())
sorted_keys = np.sort(keys)

for cval in sorted_keys:
    mean_vals = data['mean'][cval]
    std_vals = data['std'][cval]
    for j in range(n_tasks):
        colorVal = scalarMap.to_rgba(np.log(cval))
        # axs[j].plot(evals[:, j], c=colorVal)
        axs[j].errorbar(range(n_tasks), mean_vals[:, j], yerr=std_vals[:, j]/np.sqrt(n_stats), c=colorVal)
    label = "c=%g"%cval
    average = mean_vals.mean(1)  #Taking the average of cross validation accuracies accross all tasks after learning each task
    axs[-1].plot(average, c=colorVal, label=label)
    
for i, ax in enumerate(axs):
    ax.legend(loc='best')
    ax.set_title((['task %d'%j for j in range(n_tasks)] + ['average'])[i])
gcf().tight_layout()
sns.despine()